# Trying Majority vote between three methods: MOG2, CNT and Frame Differencing

In [130]:
# imports
import cv2 
import numpy as np

In [131]:
# Video Capture
cap = cv2.VideoCapture("rilevamento-intrusioni-video.wm")

In [132]:
# Init Background Subtractors
# TODO: tunning parameters different subtractors
fps = int(cap.get(cv2.CAP_PROP_FPS))
mog2 = cv2.createBackgroundSubtractorMOG2(history=5*fps, varThreshold=10, detectShadows=False)
cnt = cv2.bgsegm.createBackgroundSubtractorCNT(minPixelStability=4, useHistory=False, maxPixelStability=fps*15, isParallel=True)
ret, prev_frame = cap.read() # read first frame (for frame differencing)    

In [133]:
# Set up the window for fullscreen display
window_name = "Fullscreen Quad View"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

In [134]:
# Long Term background model
# Select the first 5 seconds of frames
frameIds = range(fps * 5)
 
# Store selected frames in an array
frames = []
for fid in frameIds:
    cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = cap.read()
    frames.append(frame)

# Calculate the median along the time axis
long_term_background_model = np.median(frames, axis=0).astype(dtype=np.uint8)    
 
# Display median frame
#cv2.imshow('frame', long_term_background_model)
#cv2.waitKey(500)
#cv2.destroyAllWindows()

In [135]:
# Processing Loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    preprocessed_frame = cv2.medianBlur(gray, 5)

    # Background Subtraction
    # MOG2
    mask_mog2 = mog2.apply(preprocessed_frame, learningRate=0.01)
    mask_mog2 = cv2.medianBlur(mask_mog2, 5)
    # CNT
    mask_cnt = cnt.apply(preprocessed_frame, learningRate=0.001)
    mask_cnt = cv2.medianBlur(mask_cnt, 5)
    # Frame Differencing
    frame_diff = cv2.absdiff(prev_frame, frame)
    mask_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    threshold, mask_diff = cv2.threshold(mask_diff, 25, 255, cv2.THRESH_BINARY)
    mask_diff = cv2.medianBlur(mask_diff, 5)

    # Update previous frame
    prev_frame = frame

    # Combined Masks
    combined_mask = ((mask_mog2 > 0).astype(np.uint8) +
                     (mask_cnt > 0).astype(np.uint8) +
                     (mask_diff > 0).astype(np.uint8))

    # Majority vote: if 2/3 masks agree, then it's a foreground pixel
    majority_vote_mask = np.where(combined_mask >= 2, 255, 0).astype(np.uint8)

    # Differenza con il background a lungo termine
    long_term_background_model_gray = cv2.cvtColor(long_term_background_model, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, long_term_background_model_gray)
    threshold_value = 25
    _, background_static_mask = cv2.threshold(diff, threshold_value, 255, cv2.THRESH_BINARY)
    background_static_mask = cv2.medianBlur(background_static_mask, 3)

    cv2.imshow("Background Static Mask", background_static_mask)

    # Combine majority vote mask with the long-term background model
    alpha = 0.4  # Weight for the majority vote mask (foreground)
    beta = 0.6   # Weight for the long-term background model (background)

    combined_with_background = cv2.addWeighted(
        majority_vote_mask.astype(np.float32), alpha,
        background_static_mask.astype(np.float32), beta,
        0
    ).astype(np.uint8)   
    
    threshold_value = 127  
    _, final_mask = cv2.threshold(
        combined_with_background, threshold_value, 255, cv2.THRESH_BINARY
    )

    #eroded = cv2.erode(final_mask,  np.ones((3, 3), np.uint8), iterations=2)

    #dilated = cv2.dilate(eroded, None, iterations=20)
    #eroded_eroded = cv2.erode(dilated,  np.ones((3, 3), np.uint8), iterations=10)

    # Stack the four images into a single image with 4 quadrants
    top_row = np.hstack((mask_mog2, mask_cnt))  # Concatenate first row
    bottom_row = np.hstack((mask_diff, majority_vote_mask))  # Concatenate second row
    
    # Concatenate the two rows vertically
    full_screen_frame = np.vstack((top_row, bottom_row))

    # Display the final fullscreen frame
    cv2.imshow(window_name, full_screen_frame)
    #cv2.imshow("Combined with Background", combined_with_background)

    contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    frame_copy = np.copy(frame)
    for contour in contours:
        #if cv2.contourArea(contour) > 50:  # Filtra i contorni troppo piccoli
            # Disegna il contorno segmentato direttamente sull'immagine
        cv2.drawContours(frame_copy, [contour], -1, (0, 255, 0), thickness=cv2.FILLED)    
    cv2.imshow("frame_copy", frame_copy)

    # Exit on ESC key
    if cv2.waitKey(30) & 0xFF == 27:
        break

    cmd = cv2.waitKey(0)
    if cmd == ord("q"):
        break

    if cmd == ord("n"):
        continue

In [136]:
# Processing Loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    preprocessed_frame = cv2.medianBlur(gray, 5)

    # Background Subtraction
    # MOG2
    mask_mog2 = mog2.apply(preprocessed_frame, learningRate=0.01)
    mask_mog2 = cv2.medianBlur(mask_mog2, 5)
    # CNT
    mask_cnt = cnt.apply(preprocessed_frame, learningRate=0.001)
    mask_cnt = cv2.medianBlur(mask_cnt, 5)
    # Frame Differencing
    frame_diff = cv2.absdiff(prev_frame, frame)
    mask_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    threshold, mask_diff = cv2.threshold(mask_diff, 25, 255, cv2.THRESH_BINARY)
    mask_diff = cv2.medianBlur(mask_diff, 5)

    # Update previous frame
    prev_frame = frame

    # Combined Masks
    combined_mask = ((mask_mog2 > 0).astype(np.uint8) +
                     (mask_cnt > 0).astype(np.uint8) +
                     (mask_diff > 0).astype(np.uint8))

    # Majority vote: if 2/3 masks agree, then it's a foreground pixel
    majority_vote_mask = np.where(combined_mask >= 2, 255, 0).astype(np.uint8)

    # Differenza con il background a lungo termine
    long_term_background_model_gray = cv2.cvtColor(long_term_background_model, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, long_term_background_model_gray)
    threshold_value = 25
    _, background_static_mask = cv2.threshold(diff, threshold_value, 255, cv2.THRESH_BINARY)
    background_static_mask = cv2.medianBlur(background_static_mask, 3)

    cv2.imshow("Background Static Mask", background_static_mask)

    # Combine majority vote mask with the long-term background model
    final_mask = np.where(
        (majority_vote_mask == 255) & (background_static_mask == 0), 255, 0
    ).astype(np.uint8)


    #eroded = cv2.erode(final_mask,  np.ones((3, 3), np.uint8), iterations=2)

    #dilated = cv2.dilate(eroded, None, iterations=20)
    #eroded_eroded = cv2.erode(dilated,  np.ones((3, 3), np.uint8), iterations=10)

    # Stack the four images into a single image with 4 quadrants
    top_row = np.hstack((mask_mog2, mask_cnt))  # Concatenate first row
    bottom_row = np.hstack((mask_diff, majority_vote_mask))  # Concatenate second row
    
    # Concatenate the two rows vertically
    full_screen_frame = np.vstack((top_row, bottom_row))

    # Display the final fullscreen frame
    cv2.imshow(window_name, full_screen_frame)
    #cv2.imshow("Combined with Background", combined_with_background)

    contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    frame_copy = np.copy(frame)
    for contour in contours:
        #if cv2.contourArea(contour) > 50:  # Filtra i contorni troppo piccoli
            # Disegna il contorno segmentato direttamente sull'immagine
        cv2.drawContours(frame_copy, [contour], -1, (0, 255, 0), thickness=cv2.FILLED)    
    cv2.imshow("frame_copy", frame_copy)

    # Exit on ESC key
    if cv2.waitKey(30) & 0xFF == 27:
        break

    cmd = cv2.waitKey(0)
    if cmd == ord("q"):
        break

    if cmd == ord("n"):
        continue

In [137]:
# Processing Loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    preprocessed_frame = cv2.medianBlur(gray, 5)

    # Background Subtraction
    # MOG2
    mask_mog2 = mog2.apply(preprocessed_frame, learningRate=0.01)
    mask_mog2 = cv2.medianBlur(mask_mog2, 5)
    # CNT
    mask_cnt = cnt.apply(preprocessed_frame, learningRate=0.001)
    mask_cnt = cv2.medianBlur(mask_cnt, 5)
    # Frame Differencing
    frame_diff = cv2.absdiff(prev_frame, frame)
    mask_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    threshold, mask_diff = cv2.threshold(mask_diff, 25, 255, cv2.THRESH_BINARY)
    mask_diff = cv2.medianBlur(mask_diff, 5)

    # Update previous frame
    prev_frame = frame

    # Combined Masks
    combined_mask = ((mask_mog2 > 0).astype(np.uint8) +
                     (mask_cnt > 0).astype(np.uint8) +
                     (mask_diff > 0).astype(np.uint8))

    # Majority vote: if 2/3 masks agree, then it's a foreground pixel
    majority_vote_mask = np.where(combined_mask >= 2, 255, 0).astype(np.uint8)

    # Differenza con il background a lungo termine
    long_term_background_model_gray = cv2.cvtColor(long_term_background_model, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, long_term_background_model_gray)
    threshold_value = 25
    _, background_static_mask = cv2.threshold(diff, threshold_value, 255, cv2.THRESH_BINARY)
    background_static_mask = cv2.medianBlur(background_static_mask, 3)

    cv2.imshow("Background Static Mask", background_static_mask)

    # Dimensione del kernel per analizzare l'intorno
    kernel_size = 7
    kernel = np.ones((kernel_size, kernel_size), dtype=np.float32)

    # Calcola la densità di foreground (majority vote)
    foreground_density = cv2.filter2D(majority_vote_mask.astype(np.float32), -1, kernel) / (kernel_size ** 2)
    foreground_density = np.clip(foreground_density, 0, 1)

    # Calcola la densità di background (statico)
    background_density = cv2.filter2D((255-background_static_mask).astype(np.float32), -1, kernel) / (kernel_size ** 2)
    background_density = np.clip(background_density, 0, 1)

    # Combina le probabilità
    final_mask = np.where(background_density > foreground_density, 255, 0).astype(np.uint8)

    # Genera la maschera finale con soglia probabilistica
    threshold_value = 0.5
    final_mask = (final_mask >= threshold_value).astype(np.uint8) * 255

    # Stack the four images into a single image with 4 quadrants
    top_row = np.hstack((mask_mog2, mask_cnt))  # Concatenate first row
    bottom_row = np.hstack((mask_diff, majority_vote_mask))  # Concatenate second row
    
    # Concatenate the two rows vertically
    full_screen_frame = np.vstack((top_row, bottom_row))

    # Display the final fullscreen frame
    cv2.imshow(window_name, full_screen_frame)
    #cv2.imshow("Combined with Background", combined_with_background)

    contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    frame_copy = np.copy(frame)
    for contour in contours:
        #if cv2.contourArea(contour) > 50:  # Filtra i contorni troppo piccoli
            # Disegna il contorno segmentato direttamente sull'immagine
        cv2.drawContours(frame_copy, [contour], -1, (0, 255, 0), thickness=cv2.FILLED)    
    cv2.imshow("frame_copy", frame_copy)

    # Exit on ESC key
    if cv2.waitKey(30) & 0xFF == 27:
        break

    cmd = cv2.waitKey(0)
    if cmd == ord("q"):
        break

    if cmd == ord("n"):
        continue

In [138]:
cap.release()
cv2.destroyAllWindows()